In [2]:
import time

start = time.time()

from bs4 import BeautifulSoup
from collections import Counter
from tldextract import extract
from mechanize import Browser 
import requests
import string


def quer_to_be_searched(r,high,dom,url):
    
    try:
       
        #CODE TO EXTRACT TITLE        
        br = Browser() 
        br.set_handle_robots(False)
        br.open(url) 
        n_title= br.title()        
        
    except:
        #TO GENERATE QUERY WITHOUT TITLE OF WEBSITE
        #print("No title found.....The website has no title")
        search=""
        for i in high:
            search += i[0]+ ' ' 
        
        z =(" ".join([dom, search]))
    
    
    else:
        #TO GENERATE QUERY WITH TITLE
        search=""
        for i in high:
            search += i[0]+ ' '
        
            
        z =(" ".join([n_title,dom, search]))
   
    return z
    
    
    
#TO COMPARE THE SITES    
def site_check(webs,query):
    c=0
    count=1
    
    while(c<=9):
                    
        try:
            url=webs[c]      
            r = requests.get(url,timeout=5)

        except:
            #print("Enough sites were not found....")
            c=10
        
        else:
            website_content = BeautifulSoup(r.text, 'html.parser').get_text()
            website_content = website_content.replace("\n"," ")
            result = calculate_frequency(website_content)
        
            k = Counter(result) 
            high = k.most_common(3) 
    
            info= extract(url)
            dom = info.domain
            
            
            a=quer_to_be_searched(r,high,dom,url)
            print("Query:" + str(count) + " " +a)
    
            count+=1
            c+=1
            
            if query==a:
                #legitimate
                c=15
                
        
    if c==15:
        return 1
    else:
        return -1                          
    
    
    
def calculate_frequency(content):
    
    stopwords= ['a', 'about', 'above', 'after', 'again', 'against', 'all',
                'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 
                'because', 'been', 'before', 'being', 'below', 'between', 
                'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 
                'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 
                'during', 'each', 'few', 'for', 'from', 'further', 'had', "hadn't", 
                'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll",
                "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself', 
                'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 
                'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 
                'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 
                'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 
                'same', "shan't", 'she', "she'd", "she'll", "she's", 'should', "shouldn't", 'so', 
                'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 
                'themselves', 'then', 'there', "there's", 'these', 'they', "they'd", "they'll", 
                "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 
                'up', 'very', 'was', "wasn't", 'we', "we'd", "we'll", "we're", "we've", 'were', "weren't", 
                'what', "what's", 'when', "when's", 'where', "where's", 'which', 'while', 'who', "who's", 
                'whom', 'why', "why's", 'with', "won't", 'would', "wouldn't", 'you', "you'd", "you'll", 
                "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
     
    
        
    new_content = ""
    for char in content:
        if char in string.punctuation:
            continue
        new_content+=char.lower()
    result = {}     

    content_list = new_content.split(" ")
    content_list = [element for element in content_list if element != ""]
    for word in content_list:
        if word in stopwords:
            continue
            
        
        if word in result.keys():
            result[word]+=1
            
        else:
            result[word]=1
            
    result = {k: v for k, v in sorted(result.items(), key=lambda item: item[1],reverse = True)} 
    return result
  

#url="https://www.axisbank.com/"
#url="https://www.onlinesbi.com/"
#url="https://www.ssl.com/article/dv-ov-and-ev-certificates/"    
#url="https://www.amazon.in/"
#url="https://www.paypal.com/in/signin"
#url="https://www.airtel.in/bank/"
#url="https://www.jiopaymentsbank.com/"

    
def content(url):
    #f = open("leg.txt", "r")
    #c=f.read()
    #if url in c:
     #   return 1
    
    result = dict()
    try:
        r = requests.get(url,timeout=30)

        website_content = BeautifulSoup(r.text, 'html.parser').get_text()
        website_content = website_content.replace("\n"," ")
        result = calculate_frequency(website_content)
        new_result = [item for item in result if not item.isdigit()]
    
        #CODE TO GET THE MOST COMMON WORDS
        k = Counter(new_result) 
        #print(k)
        high = k.most_common(3) 
    
    
        #CODE TO EXTRACT DOMAIN NAME
        info= extract(url)
        dom = info.domain

        z1=quer_to_be_searched(r,high,dom,url)
        print("Query: " + z1)
        
    
        #CODE TO GET THE SITES FROM GOOGLE SEARCH ENGINE
        try:
            from googlesearch import search 
        except ImportError:
            print("No module named 'google' found")
            return -1
  
        query = z1
        abc=""
        who=[]
        for j in search(query, tld="com", num=10,stop=10, pause=2):
            abc=j
            print(abc)
            who.append(abc)
        
        c1=site_check(who,query)
        return c1    
    
    except :
        return -1

#c=content(url)
#print(c)

end = time.time()
# total time taken
print("Runtime of the program is", end - start)

Runtime of the program is 0.0019941329956054688
